<a href="https://colab.research.google.com/github/itsanindyak/data_analysis/blob/main/text_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 27.6 MB/s eta 0:00:00


In [ ]:
!pip install -q --upgrade accelerate
!pip uninstall -q -y transformers accelerate
!pip install -q transformers accelerate

import transformers
print(transformers.__version__)


4.55.2


In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# set the model
model_bart= "facebook/bart-large-cnn"

In [ ]:
# load tokenizer
from transformers import AutoTokenizer
tokenizer= AutoTokenizer.from_pretrained(
    model_bart,
    use_fast=True
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
#load the model, Here use "AutoModelForSeq2SeqLM" b'coz model is encoder-decoder based
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_bart).to(device)


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

dataset = load_dataset("knkarthick/samsum")

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

In [ ]:
dataset['train'][1]['dialogue']

'Olivia: Who are you voting for in this election? \nOliver: Liberals as always.\nOlivia: Me too!!\nOliver: Great'

In [ ]:
length = {i : len(dataset[i]) for i in dataset}

feature = {i :dataset[i].column_names for i in dataset}

print(f"split length {length}")
print(f"feature = {feature}")
# print("\n>>100th dialogue\n")
# print(dataset['train'][100]['dialogue'])
# print("\n>>100th summary\n")
# print(dataset['train'][100]['summary'])

split length {'train': 14732, 'validation': 818, 'test': 819}
feature = {'train': ['id', 'dialogue', 'summary'], 'validation': ['id', 'dialogue', 'summary'], 'test': ['id', 'dialogue', 'summary']}


In [ ]:
def filter_empty(example):
    return example["dialogue"] is not None and example["summary"] is not None and \
           example["dialogue"].strip() != "" and example["summary"].strip() != ""

dataset = dataset.filter(filter_empty)

length = {i : len(dataset[i]) for i in dataset}
print("length afte filter",length)

Filter:   0%|          | 0/14731 [00:00<?, ? examples/s]

Filter:   0%|          | 0/818 [00:00<?, ? examples/s]

Filter:   0%|          | 0/819 [00:00<?, ? examples/s]

length afte filter {'train': 14731, 'validation': 818, 'test': 819}


In [ ]:
def convert_to_token(batch):
    encoding = tokenizer(
        batch["dialogue"],              # <-- list of strings
        text_target=batch["summary"],   # <-- list of strings
        max_length=1024,
        truncation=True,
        padding="max_length"
    )

    return encoding


In [ ]:
tokenize_dataset = dataset.map(convert_to_token,batched=True)
tokenize_dataset['train']


Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14731
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator =  DataCollatorForSeq2Seq(tokenizer,model)

In [ ]:
from transformers import TrainingArguments


arg = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",   # <-- use steps instead of epoch
    eval_steps=100,                # evaluate every 100 steps (tune this number)
    save_strategy="steps",         # also save checkpoints at same step interval
    save_steps=100,                # match eval steps for consistency
    per_device_train_batch_size=1, # fine for testing
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_steps=50,              # log training loss every 50 steps
    # load_best_model_at_end=True,   # reload best model according to metric
    # metric_for_best_model="accuracy", # or f1/loss depending on your compute_metrics
)


In [ ]:
from transformers import Trainer

trainer=Trainer(model=model,
                args=arg,
                data_collator=data_collator,
                tokenizer=tokenizer,
                train_dataset=tokenize_dataset["test"],
                eval_dataset=tokenize_dataset["validation"])

/tmp/ipython-input-1234985004.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(model=model,


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
